In [1]:
import pandas as pd
import os

# Path to the merged CSV
file_path = os.path.expanduser("~/Desktop/pems_5min_combined.csv")

# Read the CSV file
df = pd.read_csv(file_path)

print("✅ File loaded successfully")
print("🔍 Shape of dataset:", df.shape)
print("🔍 Columns:", df.columns.tolist())


✅ File loaded successfully
🔍 Shape of dataset: (59708199, 52)
🔍 Columns: ['Timestamp', 'StationID', 'District', 'Freeway', 'Direction', 'LaneType', 'StationLength', 'Samples', 'PercentObserved', 'TotalFlow', 'AvgOccupancy', 'AvgSpeed', 'Lane1_Samples', 'Lane1_Flow', 'Lane1_Occupancy', 'Lane1_Speed', 'Lane1_Observed', 'Lane2_Samples', 'Lane2_Flow', 'Lane2_Occupancy', 'Lane2_Speed', 'Lane2_Observed', 'Lane3_Samples', 'Lane3_Flow', 'Lane3_Occupancy', 'Lane3_Speed', 'Lane3_Observed', 'Lane4_Samples', 'Lane4_Flow', 'Lane4_Occupancy', 'Lane4_Speed', 'Lane4_Observed', 'Lane5_Samples', 'Lane5_Flow', 'Lane5_Occupancy', 'Lane5_Speed', 'Lane5_Observed', 'Lane6_Samples', 'Lane6_Flow', 'Lane6_Occupancy', 'Lane6_Speed', 'Lane6_Observed', 'Lane7_Samples', 'Lane7_Flow', 'Lane7_Occupancy', 'Lane7_Speed', 'Lane7_Observed', 'Lane8_Samples', 'Lane8_Flow', 'Lane8_Occupancy', 'Lane8_Speed', 'Lane8_Observed']


In [2]:
# Drop completely empty rows (all NaNs)
df.dropna(how='all', inplace=True)

# Drop rows where even lane flow/speed data is all missing
lane_speed_cols = [f'Lane{i}_Speed' for i in range(1, 9)]
df.dropna(subset=lane_speed_cols, how='all', inplace=True)

print("✅ Empty rows removed. New shape:", df.shape)


✅ Empty rows removed. New shape: (36428040, 52)


In [3]:
# Option 1: Fill missing numeric values with 0
df.fillna(0, inplace=True)

# Option 2 (Better): Use mean fill only for numeric lane metrics
# You can skip if Option 1 is enough

# for col in lane_speed_cols:
#     df[col].fillna(df[col].mean(), inplace=True)

print("✅ Missing values handled")


✅ Missing values handled


In [5]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Remove rows where timestamp couldn't be parsed
df.dropna(subset=['Timestamp'], inplace=True)

print("✅ Timestamp converted. Date range:")
print(df['Timestamp'].min(), "to", df['Timestamp'].max())


✅ Timestamp converted. Date range:
2025-01-01 00:00:00 to 2025-03-01 23:55:00


In [11]:
df = df[df['LaneType'] == 'ML']


In [12]:
final_columns = [
    'Timestamp', 'StationID', 'Freeway', 'Direction', 'LaneType',
    'TotalFlow', 'AvgOccupancy', 'AvgSpeed'
]

df_cleaned = df[final_columns]


In [13]:
# Remove rows with any negative speed or flow
df_cleaned = df_cleaned[(df_cleaned['AvgSpeed'] >= 0) & (df_cleaned['TotalFlow'] >= 0)]

# Remove extremely high occupancy (> 1 is invalid)
df_cleaned = df_cleaned[df_cleaned['AvgOccupancy'] <= 1]


In [14]:
df_cleaned.sort_values(by='Timestamp', inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)


In [15]:
final_path = os.path.expanduser("~/Desktop/pems_5min_cleaned.csv")
df_cleaned.to_csv(final_path, index=False)
print("✅ Final cleaned CSV saved at:", final_path)


✅ Final cleaned CSV saved at: /Users/spartan/Desktop/pems_5min_cleaned.csv


In [16]:
print("✅ Empty rows removed. New shape:", df_cleaned.shape)

✅ Empty rows removed. New shape: (36399420, 8)
